In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In a $M/M/1$ queue system with system load $\rho$ and capacity $\mu$ per server the arrival rate is given by $\lambda = \rho \mu$.  In the quilibrium limit of $t \rightarrow \infty$ the average number of people in the queue is given by 
$$
E[L^q] = \frac{\rho^2}{1 - \rho}.
$$
Using Little's law as 
$$
E[W] = \frac{1}{\lambda} E[L^q],
$$
we can infer the average waiting  time 
$$
E[W] = \frac{1}{\lambda} \frac{\rho^2}{1 - \rho} = \frac{\rho}{\mu (1 - \rho)}.
$$
(Adan 33)

In the case of a $M/M/n$ queue system with $n$ servers and the same system load $\rho$ and capacity $\mu$ the arrival rate is given by $\lambda = n \rho \mu$. The average number of people waiting in the queue is given by 
$$
E[L^q] = \Pi_W(n, \rho) \frac{\rho}{1 - \rho},
$$
with 
$$
\Pi_W(n, \rho) = \frac{(n \rho)^n}{n!} \left[ (1-\rho) \sum_{i=0}^{n-1} \frac{(n \rho)^i}{i!} + \frac{(n \rho)^n}{n!} \right]^{-1}.
$$
Using Little's law again now gives us
$$
E[W]  = \frac{1}{\lambda} \Pi_W(n, \rho) \frac{\rho}{1 - \rho} = \Pi_W(n, \rho) \frac{1}{n \mu} \frac{1}{1 - \rho}.
$$

The average waiting time with $\rho = 0.9$ and $\mu=1$ is equal for methods $M/M/1$ and $M/M/n=1$. This is to be expected since $\Pi_W(1, \rho) = \rho$. For higher values of $n$ the average waiting time drops drasticly for $M/M/n$ queues dispite having an identical server load. Both the $\Pi_W(n, \rho)$ and $\frac{1}{n}$ terms contribute to this decrease in waiting time. Intuitively we can imagine that  ???.

<!-- \begin{tabular}{rr}
\toprule
 n &     E[W] \\
\midrule
 1 & 9.000000 \\
 2 & 4.263158 \\
 3 & 2.723537 \\
 4 & 1.969383 \\
 5 & 1.524986 \\
 6 & 1.233543 \\
 7 & 1.028510 \\
 8 & 0.876916 \\
 9 & 0.760595 \\
10 & 0.668732 \\
\bottomrule
\end{tabular} -->



In [74]:
rho = 0.9
mu = 1

def EW_MM1(rho, mu):
    return rho / (mu * (1 - rho))

def EW_MMn(rho, mu, n):
    pi_w = Pi_w(n, rho)
    return pi_w / (n * mu) / (1 - rho)

def Pi_w(n, rho):
    b = B(n - 1, n * rho)
    return rho * b / (1 - rho + rho * b)

def B(n, rho):
    B = 1
    for i in range(1, n + 1):
        B = rho * B / (i + rho * B)
    return B

In [75]:
x = np.linspace(1, 10, 10, dtype=int)
y = [EW_MMn(rho, mu, i) for i in x]

df = pd.DataFrame({
    'n': x,
    'E[W]': y
})
print(df.to_latex(index=False))
    

\begin{tabular}{rr}
\toprule
 n &     E[W] \\
\midrule
 1 & 9.000000 \\
 2 & 4.263158 \\
 3 & 2.723537 \\
 4 & 1.969383 \\
 5 & 1.524986 \\
 6 & 1.233543 \\
 7 & 1.028510 \\
 8 & 0.876916 \\
 9 & 0.760595 \\
10 & 0.668732 \\
\bottomrule
\end{tabular}

